### using all packages

In [ ]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

In [ ]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

### using module

In [ ]:
using HighDimMixedModels

In [ ]:
#include("HighDimMixedModels.jl")
#import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [ ]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
#df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
#f = @formula(y ~  a + b + c + d)
#contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)
#df

In [ ]:
#MixedModels.datasets()
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

In [ ]:
#f = @formula(period ~ 0 + herd + incid + hsz)
#f = @formula(period ~ 0 +(1|herd) + incid + hsz )
#f = @formula(period ~ herd + incid + hsz)
#contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

### Construct object

#### 1.Number of variables indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

#### 2.indices indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

In [ ]:
HMM = highDimMixedModel(f, df, contrasts, 2, 3, 1)

#### 3.variable names indicates M,X,Z

In [ ]:
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")
HMM = highDimMixedModel(f, df, contrasts, "incid", "hsz", "herd")

In [ ]:
first(HMM.M.M,5)

In [ ]:
first(HMM.X.X,5)

In [ ]:
size(HMM.Z.Z)

In [ ]:
y, pred = modelcols(form, df);
terms = form.rhs.terms
M = highDimMat(modelmatrix(terms[idOfHDM],df))
X = XMat(modelmatrix(terms[idOfXMat],df))

#preTerms = Vector(1:length(form.rhs.terms))
#idOfReMat = preTerms[preTerms .∉ Ref(vcat(idOfHDM, idOfXmat))]
Z = ReMat(modelmatrix(terms[idOfReMat],df))

#### 4. parse random effect in formula

In [ ]:
f = @formula(period ~ 0 +(1|herd) + incid + hsz )
HMM = highDimMixedModel(f, df, 1)

In [ ]:
first(HMM.M.M,5)

In [ ]:
first(HMM.X.X,5)

In [ ]:
size(HMM.Z.Z)

### Optimization

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$
To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [ ]:
#include("HighDimMixedModels.jl")
#import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "incid", "hsz","herd")

In [ ]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

In [ ]:
HMM.optsum

In [ ]:
opt

In [ ]:
display(HMM)

In [ ]:
opt

In [ ]:
show(opt)

In [ ]:
display(opt)

In [ ]:
sigma

After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [ ]:
beta

In [ ]:
data = MixedModels.dataset(:cbpp)

In [ ]:
using CSV

In [ ]:
CSV.write("cbpp",data)

In [ ]:
pwd

In [ ]:
isnothing(12)

In [ ]:
Type{<:MixedModel}

In [ ]:
using Pkg

In [ ]:
print(Pkg.status())